In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
paths = (
    'data\\run1\\hits-0.csv',
    'data\\run1\\hits-1.csv',
    'data\\dir1\\run2\\hits-2.csv',
    'data\\dir1\\run2\\hits-3.csv',
    'data\\dir1\\run3\\hits-4.csv',
    'data\\dir1\\run3\\hits-5.csv',
)

dfs = [pd.read_csv(path, skiprows=1) for path in paths]

In [ ]:
for path, df in zip(paths, dfs):
    print(path, ':')
    print(df.head(), '\n')

In [ ]:
# Geometry Dimensions
tubeMiddleZ = 8 / 2
# Histogram/Analysis Parameters
fullZlims = (-50, 80)
numZbins = 10 * (80 + 50)  # 10 per mm
zBins = np.linspace(fullZlims[0], fullZlims[1], numZbins + 1)

# Output directory for results (histogram sums, etc.).
outDirectory = os.path.normpath('data/testcalcs_output')

## Event Testing

In [ ]:
path = paths[0]
df = dfs[0]
dfMiddle = df[df.z.abs() < tubeMiddleZ]

print(path)
print('Total energy deposit:', df.energy_deposit.sum())
print('Middle tube energy deposit:', dfMiddle.energy_deposit.sum())

In [ ]:
counts, bins = np.histogram(df.z, bins=zBins, weights=df.energy_deposit)
print(bins)
print(counts)
print(sum(counts) - df.energy_deposit.sum())  # Difference should be 0.
np.savetxt(os.path.join(outDirectory, 'event-hist.csv'), counts)

## Run Testing

In [ ]:
# More Histogram Parameters
xyLims = 7
numXYbins = 10 * 2 * xyLims  # 10 per mm
xyBins = np.linspace(-xyLims, xyLims, numXYbins + 1)
print(xyBins)

In [ ]:
eDeps, middleEdeps = [], []
zSums, xySums = [], []
for path, df in zip(paths[:2], dfs[:2]):
    print(path)
    # Energy sums.
    dfMiddle = df[df.z.abs() < tubeMiddleZ]
    edep = df.energy_deposit.sum()
    middleEdep = dfMiddle.energy_deposit.sum()
    eDeps.append(edep)
    middleEdeps.append(middleEdep)
    # Histograms.
    zSums.append(np.histogram(
        df.z, 
        bins=zBins, 
        weights=df.energy_deposit
    )[0])
    xySums.append(np.histogram2d(
        dfMiddle.x, 
        dfMiddle.y, 
        bins=2 * (xyBins,), 
        weights=dfMiddle.energy_deposit
    )[0])
    
# Energy sums.

eDeps, middleEdeps = np.array(eDeps), np.array(middleEdeps)
meanEdep, meanMiddleEdep = np.mean(eDeps), np.mean(middleEdeps)
eDepSigma = np.sqrt(np.mean((eDeps - meanEdep)**2))
middleEdepSigma = np.sqrt(np.mean((middleEdeps - meanMiddleEdep)**2))

print('Mean total energy deposit:', meanEdep)
print('Total energy deposit standard deviation:', eDepSigma)
print('Mean middle tube energy deposit:', meanMiddleEdep)
print('Middle tube energy deposit standard deviation:', middleEdepSigma)

# Histograms.
zSums, xySums = np.array(zSums), np.array(xySums)
zMeanSums, xyMeanSums = np.mean(zSums, axis=0), np.mean(xySums, axis=0)
zSumsSigmas, xySumsSigmas = np.std(zSums, axis=0), np.std(xySums, axis=0)
np.savetxt(os.path.join(outDirectory, 'run-zhist.csv'), zMeanSums)
np.savetxt(os.path.join(outDirectory, 'run-zhistsigmas.csv'), zSumsSigmas)
np.savetxt(os.path.join(outDirectory, 'run-xyhist.csv'), xyMeanSums)
np.savetxt(os.path.join(outDirectory, 'run-xyhistsigmas.csv'), xySumsSigmas.flatten())